In [1]:
import SimpleITK as sitk
import tkinter as tk
from tkinter import filedialog
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
from PIL import Image
import scipy
import os
import shutil
import pandas as pd
from scipy import signal
import cv2 as cv
import sys

import sys
# Add the directory to the sys.path
sys.path.append('D:/COFO - HP/03 - Github/dvpt-at-EMPA/HappyBisons/image analysis')
# Now you can import modules from that directory
import tools.CT_modification_functions as CT_modification_functions
import tools.CT_visualization_functions as CT_visualization_functions
import tools.CT_visualization_window as CT_visualization_window
import tools.loading_bar as loadingbar
%matplotlib qt

CT_visualization_window.check_existence()

you loaded the CT visualizer window v20.11.24


In [2]:
# ========================== defining working directories ================================

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
gr_parent_dir = os.path.dirname(parent_dir)
gr_gr_parent_dir = os.path.dirname(gr_parent_dir)

print(current_dir)
print(parent_dir)
print(gr_parent_dir)
print(gr_gr_parent_dir)


input_file_path = gr_gr_parent_dir + "\\02 - Images Processing\\023 - filtering\\filtered_image-set2-echelle.mhd"
output_folder_path = current_dir

print(input_file_path)
print(output_folder_path)



d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup\031 - create mesh volume\python
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup\031 - create mesh volume
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\02 - Images Processing\023 - filtering\filtered_image-set2-echelle.mhd
d:\COFO - HP\01 PhD research\202403 - Bern skull\WP5 - bern-001\03 - Simulation Setup\031 - create mesh volume\python


In [9]:
def detect_circles(array):
    """
    for some reason, circles is a 3D matrix. But the first dimension is useless... Tehn, each row represents one circle. It has 3 columns: 0,1 are x,y for the center of the circle; 3 is the radius of the circle

    High param1 (e.g., 200): Detects only the clearest edges, which may ignore coins with shadows or faint boundaries.
    Low param1 (e.g., 50): Detects more edges, including noise, which might lead to spurious detections.
    High param2 (e.g., 50): Requires strong evidence for circles, reducing false positives but missing faint or overlapping coins.
    Low param2 (e.g., 10): Detects more circles, but many will be false positives due to noise.

    """
    rows = array.shape[0]
    circles = cv.HoughCircles(array, cv.HOUGH_GRADIENT, 1, minDist = rows / 10,
                                param1=3, param2=30,
                                minRadius=10, maxRadius=40)
    if circles is not None:
        circles = circles[0,:,:]
    return circles

def draw_circles(array,circles):
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])
            radius = i[2]
            cv.circle(array, center, radius, (255, 0, 255), 2)
        cv.imshow("detected circles", one_image_array)

def add_circles(array,circles):
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles:
            center = (circle[0], circle[1])
            radius = circle[2]
            cv.circle(array, center, radius, (255, 0, 255), 2)
    
    return array


def list_pixels_in_circle(circle_xyr):
    x = circle_xyr[0]
    y = circle_xyr[1]
    radius = int(circle_xyr[2])
    selected_pixels = []

    for t in range(-radius, radius):        # thickness of the brush (x-axis)
        for g in range(-radius, radius):    # height of the brush (y-axis)    
            if np.sqrt(t ** 2 + g ** 2) <= radius:  # make the brush round
                selected_pixels.append([int(x + t), int(y + g)])  # append list of selected pixels
    
    selected_pixels = np.array(selected_pixels)  # convert list to numpy array
    return selected_pixels
            


In [4]:
# Load the images from file
images = sitk.ReadImage(input_file_path)
smoothed_image = sitk.SmoothingRecursiveGaussian(images, sigma=0.6)
smoothed_image = CT_modification_functions.crop_images(130, "up", smoothed_image)

#transform the image into a numpy matrix. data needs to be uint8
images_array = sitk.GetArrayFromImage(smoothed_image)
modified_images_array = images_array / images_array.max() * 255
modified_images_array = modified_images_array - 95
modified_images_array[modified_images_array < 0] = 0
modified_images_array = modified_images_array / modified_images_array.max() * 255

CT_visualization_window.open_window(sitk.GetImageFromArray(modified_images_array))


Cropping image
Current size is 3422, 130, 1024
Image cropped
New size is 3422, 130, 1024


2024-11-22 13:13:17,641 - CT_window_logger - INFO - starting window setup function
2024-11-22 13:13:17,641 - CT_window_logger - WARNING - Your image uses float32 values, values have been rescaled in the plot to fit in UINT8, but still extracted true on click.
2024-11-22 13:13:21,476 - CT_window_logger - INFO - window setup function succesfull


array([], dtype=float64)

In [10]:
slice_number = images_array.shape[0]
circles = x = np.zeros((1,3))
circled_array = np.copy(modified_images_array)          #this is the array used to visualize where the circles are
mean_in_circles = np.zeros((slice_number,5))

for n in range(0,slice_number):                                     #go through all the slices
    one_image_array = circled_array[n,:,:].astype(np.uint8)
    circles = detect_circles(one_image_array)                       #detect the circles in the current slice (x,y,r)
    if circles is not None:
        sorted_circles = circles[circles[:, 0].argsort()]           
        circled_array[n,:,:] = add_circles(circled_array[n,:,:], sorted_circles)        #draw the circles on the circled_array 
        if sorted_circles.shape[0] == 5:
            sorted_circles[:,2] = sorted_circles[:,2] * 0.8                             #reduce the radius of the circles to make sure we don't measure on the edge
            for i in range(0,sorted_circles.shape[0]):                          #for every circle
                pixels = list_pixels_in_circle(sorted_circles[i,:])                 #get the pixels inside the circle
                x_coords, y_coords = pixels[:, 0], pixels[:, 1]
                circled_array[n, y_coords, x_coords] = 0
                mean =np.mean(images_array[n, y_coords, x_coords])              #calculate the mean value of these pixels inside !!image array!!!
                mean_in_circles[n,i] = mean                                     
    
    loadingbar.main(n, slice_number, bar_length=30)

print(mean_in_circles)

CT_visualization_window.open_window(sitk.GetImageFromArray(circled_array))



Progress: [##############################] 100%
[[1036.69445801 1180.19726562 1469.97033691 1744.99377441 2013.91992188]
 [1036.72717285 1180.37194824 1470.20825195 1744.29089355 2013.36853027]
 [1036.64013672 1180.70715332 1470.50109863 1743.5177002  2014.90588379]
 ...
 [1030.21826172 1181.65771484 1470.15466309 1741.59619141 2010.53112793]
 [1029.31872559 1179.30529785 1470.2611084  1741.18005371 2009.79541016]
 [1029.46569824 1178.72033691 1468.66906738 1740.89697266 2009.21240234]]


2024-11-22 13:16:48,425 - CT_window_logger - INFO - starting window setup function
2024-11-22 13:16:48,425 - CT_window_logger - WARNING - Your image uses float32 values, values have been rescaled in the plot to fit in UINT8, but still extracted true on click.
2024-11-22 13:17:29,786 - CT_window_logger - INFO - window setup function succesfull


array([], dtype=float64)

In [ ]:
final_mean = [0,0,0,0,0]

#calculating mean accross all slices

temp = mean_in_circles[:,0]
final_mean[0] = np.mean(temp[temp != 0])
temp = mean_in_circles[:,1]
final_mean[1] = np.mean(temp[temp != 0])
temp = mean_in_circles[:,2]
final_mean[2] = np.mean(temp[temp != 0])
temp = mean_in_circles[:,3]
final_mean[3] = np.mean(temp[temp != 0])
temp = mean_in_circles[:,4]
final_mean[4] = np.mean(temp[temp != 0])
print(final_mean)


[np.float64(1017.2470455851317), np.float64(1150.585132700503), np.float64(1420.0976614453252), np.float64(1675.1975566821875), np.float64(1931.5637408895411)]


In [12]:
import plotly.graph_objects as go
import plotly.offline as pyo

def set_layout(x_name, xaxis_type, y_name) :
    layout = go.Layout(
        title={
            'text': 'EMPA: MIMIC project',
            'font': {'size': 40, 'family': 'Arial Black', 'color': 'black'},
            'x': 0.5, 'xanchor': 'center'},
        xaxis=dict(
            title=x_name,
            showgrid=True,
            zeroline=True,
            showline=True,
            zerolinecolor='black',
            zerolinewidth=3,
            gridcolor='grey',
            mirror=True,
            linewidth=2,
            type=xaxis_type,
            linecolor='black',
            tickfont=dict(
                family='Arial',
                size=25,
                color='black'
            )
        ),
        yaxis=dict(
            title=y_name,
            showgrid=True,
            zeroline=True,
            showline=True,
            zerolinecolor='black',
            zerolinewidth=3,
            gridcolor='grey',
            mirror=True,
            linewidth=2,
            linecolor='black',
            tickfont=dict(
                family='Arial',
                size=25,
                color='black'
            )
        ),
        font=dict(
            family='Arial',
            size=25,
            color='black'
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        hovermode='closest',
        
    )
    return layout



fig = go.Figure()

#measured values
x_values = final_mean
y_values = np.array([802,602,400,198,102])

sample_name = "filtered_image-set1"

fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines',name=sample_name))
fig.update_traces(showlegend = True)

#linear fit
m, b = np.polyfit(y_values, x_values, 1)

y_fit = np.array(range(0,1250,100))
x_fit = m*y_fit+b

fig.add_trace(go.Scatter(x=x_fit, y=y_fit, mode='lines',name=f"linear regression; m={m}, b={b}"))
fig.update_traces(showlegend = True)

# Set up layout
layout = set_layout("grey level", "linear", "HA density")
fig.update_layout(layout)

#plot figure
pyo.plot(fig, auto_open=True)

CT_visualization_window.open_window(images)

2024-11-22 13:17:33,724 - CT_window_logger - INFO - starting window setup function
2024-11-22 13:17:33,725 - CT_window_logger - WARNING - Your image uses uint16 values, values have been rescaled in the plot to fit in UINT8, but still extracted true on click.
2024-11-22 13:17:40,686 - CT_window_logger - INFO - window setup function succesfull


array([], dtype=float64)